<a href="https://colab.research.google.com/github/ArvindReddyC/Sord_analysis/blob/main/prediction_SORD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras
from tensorflow import keras
from scipy import stats
from tqdm import tqdm
from iterto

In [ ]:
!touch test.py

# Function Required.

In [11]:
def encapsule(df):
    display(df.head(5))
    print( f'shape of df = {df.shape}' )
    return df

In [12]:
def tran(x):
    y = x[1:-1]
    return y.split(',')

def data_wrangle( df ):
    sample = df[['timestamp','raw_sensor_data']];
    sample['parameters'] = sample.raw_sensor_data.apply(tran);
    main = pd.DataFrame( sample.parameters.to_list() , columns = ['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11'] );
    main['timestamp'] = sample['timestamp'];
    main.drop( 'v1' , axis = 1 , inplace = True );
    main.set_index('timestamp' , inplace=True);
    return main;

def data_wrangle_json( df ):
    main = pd.DataFrame( df.raw_sensor_data.to_list() , columns = ['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11'] );
    main['timestamp'] = df['timestamp'];
    main.drop( 'v1' , axis = 1 , inplace = True );
    main.set_index('timestamp' , inplace=True);
    return main;

In [13]:
def create_segments_and_labels(df, time_steps, step):

    # x, y, z acceleration as features
    N_FEATURES = 10
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['v3'].values[i: i + time_steps]
        ys = df['v4'].values[i: i + time_steps]
        zs = df['v5'].values[i: i + time_steps]
        angle = df['v2'].values[i: i + time_steps]
        u5 = df['v6'].values[i: i + time_steps]
        u6= df['v7'].values[i: i + time_steps]
        u7= df['v8'].values[i: i + time_steps]
        u8= df['v9'].values[i: i + time_steps]
        u9= df['v10'].values[i: i + time_steps]
        u10= df['v11'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        segments.append([xs, ys, zs,angle,u5,u6,u7,u8,u9,u10 ])

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    reshaped = reshaped_segments.reshape(-1,350)
    return reshaped

# Predicting from CSV File 

In [5]:
sample = encapsule(pd.read_csv('new_test_data.csv'))


,id,user_id,email_id,raw_sensor_data,predicted_activity,actual_activity,timestamp,createdAt,updatedAt
0,458813,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[1.626615,8.192087525665539,-0.99,0.8,-9.53,-0...",IN_PROGRESS,NaN,2022-02-13T14:32:00.378Z,2022-02-14T01:32:45.325Z,2022-02-14T01:32:45.325Z
1,458814,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[2.114199,8.192087525665539,-0.99,0.81,-9.6,-0...",IN_PROGRESS,NaN,2022-02-13T14:32:00.866Z,2022-02-14T01:32:45.332Z,2022-02-14T01:32:45.332Z
2,458815,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[2.598735,8.192087525665539,-1.01,0.84,-9.58,-...",IN_PROGRESS,NaN,2022-02-13T14:32:01.350Z,2022-02-14T01:32:45.340Z,2022-02-14T01:32:45.340Z
3,458816,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[3.08846,8.192087525665539,-0.99,0.78,-9.57,0....",IN_PROGRESS,NaN,2022-02-13T14:32:01.840Z,2022-02-14T01:32:45.348Z,2022-02-14T01:32:45.348Z
4,458817,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[3.573518,8.192087525665539,-0.99,0.82,-9.55,0...",IN_PROGRESS,NaN,2022-02-13T14:32:02.324Z,2022-02-14T01:32:45.363Z,2022-02-14T01:32:45.363Z


shape of df = (8240, 9)


In [6]:
main = encapsule(data_wrangle(sample));


,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11
timestamp,,,,,,,,,,
2022-02-13T14:32:00.378Z,8.192087525665539,-0.99,0.8,-9.53,-0.0625,0.0625,0,16.1875,-19.375,-61.875
2022-02-13T14:32:00.866Z,8.192087525665539,-0.99,0.81,-9.6,-0.0625,0,-0.0625,15.75,-18.6875,-62.75
2022-02-13T14:32:01.350Z,8.192087525665539,-1.01,0.84,-9.58,-0.1875,-0.125,-0.25,16.5,-19.375,-62.25
2022-02-13T14:32:01.840Z,8.192087525665539,-0.99,0.78,-9.57,0.3125,-0.0625,-0.0625,16.1875,-19.0625,-62.75
2022-02-13T14:32:02.324Z,8.192087525665539,-0.99,0.82,-9.55,0.0625,0.0625,0,16.5,-19.0625,-61.875


shape of df = (8240, 10)


In [15]:
def main():
  # some_list = []
  # for each in data:
  #   some_list.append(each["data"][0])
  # sample = pd.DataFrame.from_dict(some_list)
  # main = data_wrangle_json(sample)
  sample = pd.read_csv('new_test_data.csv')
  main = data_wrangle(sample);
  reshaped = create_segments_and_labels(main,35,1)
  model = keras.models.load_model('/content/saved_model/my_model_2')
  currently_activity = ''
  number_of_windoews = len(sample) - 35
  from_ = sample.loc[0,'timestamp']
  Clsses = ['sit', 'std', 'wlk']
  final_list = []
  for each in tqdm(range(number_of_windoews)):
    y_hat =  model.predict(reshaped[each].reshape(-1,350))
    y_hat  = np.argmax(y_hat)
    y_hat = Clsses[y_hat]
    if((y_hat != currently_activity) & (y_hat != '') ):
      final_list.append( { 'start_time': from_ , 'end_time':sample.loc[each,'timestamp'] , 'activity': y_hat  } )
      currently_activity = y_hat
      from_ = sample.loc[each,'timestamp']
  return final_list


In [14]:
!tar -xvzf model_2.tar.gz

saved_model/my_model_2/
saved_model/my_model_2/saved_model.pb
saved_model/my_model_2/keras_metadata.pb
saved_model/my_model_2/assets/
saved_model/my_model_2/variables/
saved_model/my_model_2/variables/variables.index
saved_model/my_model_2/variables/variables.data-00000-of-00001


In [16]:
main()

100%|██████████| 8205/8205 [07:54<00:00, 17.29it/s]


[{'activity': 'std',
  'end_time': '2022-02-13T14:32:00.378Z',
  'start_time': '2022-02-13T14:32:00.378Z'},
 {'activity': 'wlk',
  'end_time': '2022-02-13T14:36:48.248Z',
  'start_time': '2022-02-13T14:32:00.378Z'},
 {'activity': 'std',
  'end_time': '2022-02-13T14:37:02.286Z',
  'start_time': '2022-02-13T14:36:48.248Z'},
 {'activity': 'wlk',
  'end_time': '2022-02-13T14:37:02.776Z',
  'start_time': '2022-02-13T14:37:02.286Z'},
 {'activity': 'sit',
  'end_time': '2022-02-13T14:37:04.722Z',
  'start_time': '2022-02-13T14:37:02.776Z'},
 {'activity': 'wlk',
  'end_time': '2022-02-13T14:39:21.273Z',
  'start_time': '2022-02-13T14:37:04.722Z'},
 {'activity': 'sit',
  'end_time': '2022-02-13T14:39:21.764Z',
  'start_time': '2022-02-13T14:39:21.273Z'},
 {'activity': 'wlk',
  'end_time': '2022-02-13T14:39:22.243Z',
  'start_time': '2022-02-13T14:39:21.764Z'},
 {'activity': 'sit',
  'end_time': '2022-02-13T14:39:36.973Z',
  'start_time': '2022-02-13T14:39:22.243Z'},
 {'activity': 'wlk',
  'end_

## Testing

In [ ]:
print('='*50 , 'From the Rule based' ,'='*50  )
print(repr(sample.predicted_activity.value_counts()))
print('='*50 , 'From the Model' ,'='*50  )
print(repr(Counter(preds)))

## Acquiring index where there is change in activity

In [ ]:
sample.loc[np.where(np.roll(preds,1)!=preds)[0]]['timestamp']

# Predicting all instances at once 

In [21]:
sample = pd.read_csv('new_test_data.csv')
main = data_wrangle(sample);
reshaped = create_segments_and_labels(main,35,1)
model = keras.models.load_model('/content/saved_model/my_model_2')
preds =  model.predict(reshaped)
# indices = np.argmax(preds)
# classes = []
# Clsses = ['sit', 'std', 'wlk']
# for each in indices:
#  classes.append(Clsses[each])

In [24]:
list_of_indeces = []
for each in preds:
  list_of_indeces.append(np.argmax(each))


In [25]:
Clsses = ['sit', 'std', 'wlk']
classes = []
for each in list_of_indeces:
 classes.append(Clsses[each])

In [31]:
from collections import Counter
Counter(classes)

Counter({'sit': 508, 'std': 6422, 'wlk': 1275})

# This is for Json data 

In [ ]:
import numpy as np
import pandas as pd
json = [{
  "data": [
    {
      "raw_sensor_data": list(np.random.randint(10 , size = 11)),
      "timestamp": "2022-02-10 01:01:00"    },
      {
      "raw_sensor_data": list(np.random.randint(10 , size = 11)),
      "timestamp": "2022-02-10 01:02:00"    },
      {
      "raw_sensor_data": list(np.random.randint(10 , size = 11)),
      "timestamp": "2022-02-10 01:03:00"    },
      {
      "raw_sensor_data": list(np.random.randint(10 , size = 11)),
      "timestamp": "2022-02-10 01:04:00"    }
  ],
  "mode": "DEVICE",
  "location": {
    "latitude": 0,
    "longitude": 0  },
  "deviceStatus": "CONNECTED",
  "actualActivity": "SITTING"}]


json[0]['data']
sample = pd.DataFrame.from_dict(json[0]['data'])
sample

,raw_sensor_data,timestamp
0,"[4, 8, 4, 0, 6, 0, 7, 1, 4, 9, 9]",2022-02-10 01:01:00
1,"[1, 9, 5, 2, 3, 0, 6, 0, 9, 4, 5]",2022-02-10 01:02:00
2,"[9, 0, 2, 9, 5, 7, 2, 1, 0, 5, 6]",2022-02-10 01:03:00
3,"[3, 5, 8, 4, 4, 1, 7, 1, 7, 1, 1]",2022-02-10 01:04:00


In [ ]:
def main(data):
  some_list = []
  for each in data:
    some_list.append(each["data"][0])
  sample = pd.DataFrame.from_dict(some_list)
  main = data_wrangle_json(sample)
  # sample = pd.read_csv('sord.csv')
  # main = data_wrangle(sample);
  reshaped = create_segments_and_labels(main,35,1)
  model = keras.models.load_model('/content/saved_model/my_model_2')
  currently_activity = ''
  number_of_windoews = len(sample) - 35
  from_ = sample.loc[0,'timestamp']
  Clsses = ['sit', 'std', 'wlk']
  final_list = []
  for each in tqdm(range(number_of_windoews)):
    y_hat =  model.predict(reshaped[each].reshape(-1,350))
    y_hat  = np.argmax(y_hat)
    y_hat = Clsses[y_hat]
    if((y_hat != currently_activity) & (y_hat != '') ):
      final_list.append( { 'start_time': from_ , 'end_time':sample.loc[each,'timestamp'] , 'activity': y_hat  } )
      currently_activity = y_hat
      from_ = sample.loc[each,'timestamp']
  return final_list

saved_model/my_model_2/
saved_model/my_model_2/saved_model.pb
saved_model/my_model_2/keras_metadata.pb
saved_model/my_model_2/assets/
saved_model/my_model_2/variables/
saved_model/my_model_2/variables/variables.index
saved_model/my_model_2/variables/variables.data-00000-of-00001


In [ ]:
repr(final_list)

"[{'start_time': '2022-02-02T16:13:39.218Z', 'end_time': '2022-02-02T16:13:39.218Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:13:39.218Z', 'end_time': '2022-02-02T16:23:55.463Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:23:55.463Z', 'end_time': '2022-02-02T16:23:56.437Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:23:56.437Z', 'end_time': '2022-02-02T16:23:57.412Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:23:57.412Z', 'end_time': '2022-02-02T16:23:57.900Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:23:57.900Z', 'end_time': '2022-02-02T16:37:40.067Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:37:40.067Z', 'end_time': '2022-02-02T16:37:41.042Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:37:41.042Z', 'end_time': '2022-02-02T16:37:41.530Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:37:41.530Z', 'end_time': '2022-02-02T16:37:42.991Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:37:42.991Z', 'end_time': '2022-02-02T16:3

================================================== From the Rule based ==================================================
sit    4357
std     513
wlk     130
Name: predicted_activity, dtype: int64
================================================== From the Model ==================================================
Counter({'std': 4726, 'wlk': 239})


In [ ]:
dummy_data = [{
  "data": [
    {
      "raw_sensor_data": list(np.random.randint(10 , size = 11)),
      "timestamp": "2022-02-10"    }
  ],
  "mode": "MANUAL",
  "location": {
    "latitude": 0,
    "longitude": 0  },
  "deviceStatus": "CONNECTED",
  "actualActivity": "SITTING"} , {
  "data": [
    {
      "raw_sensor_data": list(np.random.randint(10 , size = 11)),
      "timestamp": "2022-02-10"    }
  ],
  "mode": "MANUAL",
  "location": {
    "latitude": 0,
    "longitude": 0  },
  "deviceStatus": "CONNECTED",
  "actualActivity": "SITTING"}] 

In [ ]:
data_wrangle_json(pd.DataFrame.from_dict(some_list))

,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11
timestamp,,,,,,,,,,
2022-02-10,0,0,3,8,7,9,4,6,1,5
2022-02-10,2,8,8,5,8,9,9,2,9,1


In [ ]:
Predict_a_block(block-of-data):

current_activity = nil



from = starttime (from the data)

timewindow array = [ from, to, activity]



for: how many sliding windows of 35 in the block-of-data -- len(b)

slice = window

prediction = model_m.predict(slice)

if (prediction != current_activity):

if (current_activity != nil):

add to the timewindow array (from , currentime, current_activity)

set current_activity = prediction

set from: currenttime



return the window array

[

    {from , to , sit},

{from, to, walk},

{....}

]